In [11]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from urllib.request import urlopen
import urllib.request
from datetime import datetime
from tqdm import tqdm
import time
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed

In [12]:
hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 


In [3]:
for i in range(1,50000):

    while True:
        try:
            
            r = requests.get(f'https://bidadari.my/senarai-artikel/page/{i}/?expand_article=1',headers=headers)
            break
        except:
            time.sleep(3.0)
    

    soup = BeautifulSoup(r.text, "html.parser")

    div = soup.find_all('div', attrs = {"class":"article-post"})

    if len(div) == 0:
        break

    for link in soup.find_all('div', attrs = {"class":"article-post"}):

        href = link.find('a').get('href')

        hrefs.append(href)
                    


In [15]:
len(hrefs)

3647

In [14]:
hrefs = list(set(hrefs))

In [8]:
with open(f'bidadari-link.json', 'a') as f:
    json.dump(hrefs, f)

In [13]:
with open('bidadari-link.json') as fopen:
    hrefs = json.load(fopen)

In [16]:
def process_url(x):
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            
            if r.status_code == 508:
                time.sleep(20)
                continue
                
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "html.parser")

    try:
        headline = soup.find('div', class_="feed").text
        h = soup.find('div', class_="konten")
        p = [p.text for p in h.find_all(['p','h1','h2','h3','h4']) if p.text and len(p.text.split()) > 5]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x, 'headline': headline, 'content': p}
    
    return data

In [17]:
max_workers = 10

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('bidadari.jsonl', 'a') as f:
                json.dump(result, f)
                f.write('\n')

100%|██████████| 3647/3647 [02:55<00:00, 20.79it/s]
